<a href="https://www.kaggle.com/code/kerthi99055/safegrid-testcode?scriptVersionId=220811765" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2

# Path to the dataset
dataset_path = "/kaggle/input/rwf2000/RWF-2000"

# Function to calculate video duration
def get_video_duration(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return 0
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_count / fps if fps > 0 else 0
    cap.release()
    return duration

# Function to process dataset
def process_dataset(dataset_path):
    folder_stats = {}
    total_duration = 0

    for root, dirs, files in os.walk(dataset_path):
        video_count = 0
        folder_duration = 0
        for file in files:
            if file.endswith((".mp4", ".avi", ".mkv")): 
                video_count += 1
                video_path = os.path.join(root, file)
                folder_duration += get_video_duration(video_path)
        
        if video_count > 0:
            folder_name = os.path.basename(root)
            folder_stats[folder_name] = {
                "video_count": video_count,
                "duration": folder_duration
            }
            total_duration += folder_duration

    return folder_stats, total_duration

# Process the dataset
stats, total_duration = process_dataset(dataset_path)

# Print results
for folder, data in stats.items():
    print(f"Folder: {folder}")
    print(f"  Videos: {data['video_count']}")
    print(f"  Duration: {data['duration']:.2f} seconds\n")

print(f"Total Duration of Dataset: {total_duration:.2f} seconds")

In [ ]:
#import the neccesary libraries 
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [ ]:
# Parameters
IMG_SIZE = 64  # Resize frames to 64x64
SEQ_LENGTH = 30  # Number of frames per video
DATASET_PATH = "/kaggle/input/rwf2000/RWF-2000"

In [ ]:
# preprocess videos
def load_preprocess_videos(path, label, limit=None):
    videos = []
    labels = []
    count = 0
    for video_name in os.listdir(path):
        video_path = os.path.join(path, video_name)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while len(frames) < SEQ_LENGTH:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
            frame = frame / 255.0  
            frames.append(frame)
        cap.release()
        
        if len(frames) == SEQ_LENGTH:
            videos.append(np.array(frames))
            labels.append(label)
            count += 1
            if limit and count >= limit:
                break
    return np.array(videos), np.array(labels)


In [ ]:
def prepare_dataset():
    X_train, y_train, X_test, y_test = [], [], [], []
    for label, subfolder in enumerate(['Fight', 'NonFight']):
        train_path = os.path.join(DATASET_PATH, 'train', subfolder)
        
        # Reserve 10% of training videos as test set
        videos, labels = load_preprocess_videos(train_path, label)
        split_idx = int(0.1 * len(videos))  # 10% split
        
        X_test.extend(videos[:split_idx])
        y_test.extend(labels[:split_idx])
        X_train.extend(videos[split_idx:])
        y_train.extend(labels[split_idx:])
    
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)


In [ ]:
def build_model():
    model = Sequential([
        Conv3D(32, (3, 3, 3), activation='relu', input_shape=(SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling3D((2, 2, 2)),
        BatchNormalization(),
        Conv3D(64, (3, 3, 3), activation='relu'),
        MaxPooling3D((2, 2, 2)),
        BatchNormalization(),
        Conv3D(128, (3, 3, 3), activation='relu'),
        MaxPooling3D((2, 2, 2)),
        BatchNormalization(),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')  # Two classes: Fight, NonFight
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    
# Prepare the data
print("Loading dataset...")
X_train, y_train, X_test, y_test = prepare_dataset()
y_train = to_categorical(y_train, num_classes=2)  # Convert labels to one-hot encoding
y_test = to_categorical(y_test, num_classes=2)

In [ ]:
# Train-validation split from training set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Build and train the model
print("Building model...")
model = build_model()

print("Training model...")
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_val, y_val))

# Save the model
model.save('violence_detection_model.h5')
print("Model saved!")

In [ ]:
# Evaluate on test data
print("Evaluating on test set...")
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:

def predict_video(frames):
    frames = np.expand_dims(np.array(frames), axis=0)  
    prediction = model.predict(frames)
    classes = ['Fight', 'NonFight']
    return classes[np.argmax(prediction)]

print("Testing individual test videos...")
for i in range(5):  
    frames = X_test[i]
    actual_label = 'Fight' if np.argmax(y_test[i]) == 0 else 'NonFight'
    predicted_label = predict_video(frames)
    print(f"Actual: {actual_label}, Predicted: {predicted_label}")